In [1]:
# OpenCV
%conda install -c conda-forge opencv

# NumPy
%conda install numpy

# PyTorch (CUDA 버전에 따라 적절한 버전을 선택해야 합니다)
%conda install pytorch torchvision torchaudio cpuonly -c pytorch
# 또는 CUDA 지원을 추가하려면
# conda install pytorch torchvision torchaudio cudatoolkit=11.8 -c pytorch

# Ultralytics (YOLO)
%pip install ultralytics

# Albumentations
%pip install albumentations

# tqdm
%conda install tqdm

/bin/bash: conda: command not found

Note: you may need to restart the kernel to use updated packages.
/bin/bash: conda: command not found

Note: you may need to restart the kernel to use updated packages.
/bin/bash: conda: command not found

Note: you may need to restart the kernel to use updated packages.
  Using cached opencv_python-4.10.0.84-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (20 kB)
  Using cached py_cpuinfo-9.0.0-py3-none-any.whl.metadata (794 bytes)
  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached ultralytics_thop-2.0.0-py3-none-any.whl.metadata (8.5 kB)
  Using cached pytz-2024.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.1-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached filelock-3.15.4-py3-none-any.whl.metadata (2.9 kB)
  Using cached networkx-3.3-py3-none-any.whl.metadata (5.1 kB)
  Using cached jinja2-3.1.4-py3-none-any.whl.metadata (2.6 kB)
  Using cached fsspec-2024.6.1-py3-none-any.w

In [ ]:
import os
import json
import cv2
import numpy as np
import torch
from ultralytics import YOLO
from albumentations import (
    Compose, RandomBrightnessContrast, HueSaturationValue, GaussNoise,
    HorizontalFlip, RandomCrop, Rotate, ShiftScaleRotate
)
from torch.utils.data import DataLoader
from tqdm import tqdm

# Data loading and preprocessing function
def load_and_preprocess_data(json_path, image_dir):
    with open(json_path, 'r') as f:
        data = json.load(f)
    
    image_path = os.path.join(image_dir, data['image_name'])
    
    if not os.path.exists(image_path):
        print(f"Image file not found: {image_path}")
        return None, None, None, None
    
    image = cv2.imread(image_path)
    
    if image is None:
        print(f"Failed to load image: {image_path}")
        return None, None, None, None
    
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    annotations = data['Annotation']
    boxes = []
    labels = []
    for ann in annotations:
        boxes.append(ann['data'])
        labels.append(ann['class_name'])
    
    return image, np.array(boxes), labels, data['image_name']

# Data augmentation function
def augment_data(image, boxes, labels):
    augmentations = Compose([
        RandomBrightnessContrast(p=0.5),
        HueSaturationValue(p=0.5),
        GaussNoise(p=0.3),
        HorizontalFlip(p=0.5),
        RandomCrop(height=image.shape[0], width=image.shape[1], p=0.3),
        Rotate(limit=10, p=0.3),
        ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=10, p=0.3)
    ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})
    
    augmented = augmentations(image=image, bboxes=boxes, labels=labels)
    return augmented['image'], np.array(augmented['bboxes']), augmented['labels']

# Convert to YOLO format
def convert_to_yolo_format(boxes, labels, image_size):
    yolo_labels = []
    for box, label in zip(boxes, labels):
        x_center = (box[0] + box[2]) / 2 / image_size[1]
        y_center = (box[1] + box[3]) / 2 / image_size[0]
        width = (box[2] - box[0]) / image_size[1]
        height = (box[3] - box[1]) / image_size[0]
        class_id = class_mapping[label]
        yolo_labels.append(f"{class_id} {x_center} {y_center} {width} {height}")
    return yolo_labels

# Prepare dataset function
def prepare_dataset(data_dir):
    images_dir = os.path.join(data_dir, 'images')
    labels_dir = os.path.join(data_dir, 'labels')
    
    data = []
    for json_file in os.listdir(labels_dir):
        if json_file.endswith('.json'):
            json_path = os.path.join(labels_dir, json_file)
            image, boxes, labels, image_name = load_and_preprocess_data(json_path, images_dir)
            
            if image is None:
                continue
            
            # Original image and label processing
            orig_yolo_labels = convert_to_yolo_format(boxes, labels, image.shape[:2])
            orig_label_filename = f"{json_file[:-5]}.txt"
            orig_label_path = os.path.join(labels_dir, orig_label_filename)
            with open(orig_label_path, 'w') as f:
                f.write('\n'.join(orig_yolo_labels))
            
            # Augmented data
            aug_image, aug_boxes, aug_labels = augment_data(image, boxes, labels)
            aug_yolo_labels = convert_to_yolo_format(aug_boxes, aug_labels, aug_image.shape[:2])
            
            # Store both original and augmented data
            data.append((image_name, orig_label_path, aug_image, aug_yolo_labels))
    
    return data

# Class mapping definition
class_mapping = {
    'car': 0, 'bus': 1, 'truck': 2, 'special vehicle': 3,
    'motorcycle': 4, 'bicycle': 5, 'personal mobility': 6,
    'person': 7, 'Traffic_light': 8, 'Traffic_sign': 9
}

# Custom dataset class
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        image_path, label_path, aug_image, aug_labels = item
        
        if aug_image is not None:  # Augmented data
            image = cv2.cvtColor(aug_image, cv2.COLOR_BGR2RGB)
            labels = aug_labels
        else:  # Original image
            image = cv2.imread(image_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            with open(label_path, 'r') as f:
                labels = f.read().splitlines()
        
        # Further preprocessing can be done here if needed
        
        return image, labels

# Main execution
if __name__ == "__main__":
    print("Current working directory:", os.getcwd())
    
    # Prepare dataset
    train_data = prepare_dataset('training')
    val_data = prepare_dataset('validation')
    
    print(f"Prepared {len(train_data)} training samples")
    print(f"Prepared {len(val_data)} validation samples")

    # Create custom datasets
    train_dataset = CustomDataset(train_data)
    val_dataset = CustomDataset(val_data)
    
    # DataLoader for batching
    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=4)
    val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=4)
    
    # YAML config file
    yaml_content = f"""
    train: training/images
    val: validation/images
    test: test/images

    nc: {len(class_mapping)}
    names: {list(class_mapping.keys())}
    """

    with open('dataset.yaml', 'w') as f:
        f.write(yaml_content)

    # Load YOLOv5 model
    model = YOLO('yolov5n.pt')

    # Model training
    results = model.train(
        data='dataset.yaml',
        epochs=20,
        imgsz=640,
        batch=16,
        save_period=2,  # Save model every 2 epochs
        weight_decay=0.005,  # L2 regularization
        mosaic=0.5,  # Mosaic augmentation
        mixup=0.2,  # Mixup augmentation
        name='yolov5n_autonomous_driving',
        early_stopping=True,  # Enable early stopping
        patience=5,  # Patience for early stopping
        save_best_only=True  # Save only the best model
    )

    # Model evaluation
    results = model.val()

    # Test data inference
    test_results = model('test/images')

    # Save test results
    test_results.save('test_results')

    # Export model (ONNX format)
    model.export(format='onnx', simplify=True, opset=13, dynamic=True, half=True)
